In [42]:
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Trade_Functions import *


In [43]:
cal = get_calendar()

In [51]:
def pick_trade(RunDate,NumStocks,cal=get_calendar(),holdTime=1,TimeInput=5,NDXfile="Nasdaq100MembersJan2024.csv"):
    LongStockList = list()
    ShortStockList = list()
    # Prior day input for the trading algo
    InputDate = cal[cal.index(RunDate)-1]
    PctChange = get_SortedPctChange(InputDate,TimeInput,NDXfile)
    
    print(PctChange)

    # Now that we have the sorted Returns, lets trade the top and bottom performers.
    # The idea is to play momentum. Lets go long the top x names and short the bottom x names
    # Idea is that we put the trade on at the beginning of the day...so we get Open price and close out at
    # the end of day at the Close price.
    LongStockList = PctChange["Ticker"].iloc[:NumStocks]
    ShortStockList =  PctChange["Ticker"].iloc[len(PctChange["Ticker"])-NumStocks:]
       
    
    
    # Lets get the return of this Long/short portfolio over the day
    
    LongReturns = getLongReturns(RunDate,LongStockList,holdTime)
    ShortReturns = getShortReturns(RunDate,ShortStockList,holdTime)
    
    print("Long Return:", LongReturns)
    print("Short Return:", ShortReturns)

    
    TotalReturn = (sum(LongReturns) + sum(ShortReturns))/NumStocks

    print("Total Return: ", TotalReturn)
    
    return TotalReturn

In [50]:
g =pick_trade('2024-11-05',3)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Total Return:  -0.0012865750899363694


In [56]:
g =pick_trade('2024-09-11',3)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

    Ticker  PctChange
0     MRNA   0.093668
1     IDXX   0.052324
2      ARM   0.051840
3     FAST   0.047917
4     MNST   0.040306
..     ...        ...
96     AZN  -0.053528
97    NXPI  -0.057230
98    CPRT  -0.066730
99    FANG  -0.069405
100   ASML  -0.074062

[101 rows x 2 columns]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Long Return: [0.041513346278179625, 0.03084539358422309, -0.006443023624817182]
Short Return: [-0.0018211282056348839, 0.041044546567890264, -0.0019602776212151118]
Total Return:  0.0343929523262086


In [59]:
def getLongReturns(RunDate,LongStockList,holdTime=1):
    LongReturnList = list()
    cal = get_calendar()
    
    for ticker in LongStockList:
        
        endDate = cal[cal.index(RunDate)+holdTime]

        #Get the data based on the provided ticker, RunDate, and startDate(being runDate-Timeinput)
        data = (yf.download(ticker, start=RunDate, end=endDate,multi_level_index=False)).reset_index()
        print(data)
        #Get the Close and Open Values
        ClosePrice=data["Close"].values
        OpenPrice = data['Open'].values
        
        #Calculate and Append Return
        Return = (ClosePrice[-1]-OpenPrice[0])/OpenPrice[0]
        LongReturnList.append(Return)
    return LongReturnList

In [60]:
getLongReturns('2024-09-11',["MRNA"])

[*********************100%***********************]  1 of 1 completed

                       Date  Adj Close      Close       High        Low  \
0 2024-09-11 00:00:00+00:00  79.510002  79.510002  79.879997  77.440002   

        Open   Volume  
0  79.050003  3370700  


[0.005819090028010156]

In [45]:
TimeInput = 5
NDXfile="Nasdaq100MembersJan2024.csv"
NumStocks=3

In [47]:
TotalReturnList = list()
DateReturnList = list()

for day in cal[TimeInput+1:]:
    dayReturn = pick_trade(RunDate=day,NumStocks=NumStocks,cal=Cal,TimeInput=TimeInput,NDXfile=NDXfile)
    TotalReturnList.append(dayReturn)
    DateReturnList.append(day)

dfOut = pd.DataFrame({'Date':DateReturnList,'Return':TotalReturnList})







[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Total Return:  0.01927848126618753


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Total Return:  0.005188068544672921


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

Total Return:  -0.0031155246060469923


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Total Return:  0.00483319708066657


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Total Return:  0.01880355730352926


In [49]:
dfOut

,Date,Return
0,2024-01-10,-0.009347
1,2024-01-11,-0.009347
2,2024-01-12,-0.009347
3,2024-01-16,-0.009347
4,2024-01-17,-0.009347
...,...,...
206,2024-11-04,-0.009347
207,2024-11-05,-0.009347
208,2024-11-06,-0.009347
209,2024-11-07,-0.009347


In [48]:
dfOut.to_csv("ReturnData.csv")
